In [0]:
from pyspark.sql.functions import col, year, month, to_date

In [0]:
reviews_with_time = spark.table("workspace.default.silverreview") \
    .withColumn("date_parsed", to_date(col("date"))) \
    .withColumn("year", year(col("date_parsed"))) \
    .withColumn("month", month(col("date_parsed")))

In [0]:
gold_master_reviews = reviews_with_time.alias("r") \
    .join(spark.table("workspace.default.silverbusiness").alias("b"), 
          col("r.business_id") == col("b.business_id"), "left") \
    .join(spark.table("workspace.default.silveruser").alias("u"), 
          col("r.user_id") == col("u.user_id"), "left") \
    .select("r.*", "b.city", "b.categories", "u.review_count", "u.yelping_since")

In [0]:
gold_master_reviews.display

In [0]:
gold_master_reviews.write.format("delta").mode("overwrite") \
    .saveAsTable("workspace.default.gold_review_master")